In [1]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [2]:
audio_train=pd.read_csv('Temp_test_audio.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,217,136,NaN,1b1,Ar,sc,Meditron,9,9,0,0,0,136_1b1_Ar_sc_Meditron.txt,test
1,247,139,NaN,1b1,Al,sc,Litt3200,10,10,0,0,0,139_1b1_Al_sc_Litt3200.txt,test
2,248,139,NaN,1b1,Ar,sc,Litt3200,9,9,0,0,0,139_1b1_Ar_sc_Litt3200.txt,test
3,249,139,NaN,1b1,Ll,sc,Litt3200,8,7,0,0,1,139_1b1_Ll_sc_Litt3200.txt,test


In [3]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=50
highcut=2000

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [5]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [6]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [7]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [19]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = []
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            print("data:-")
            #print(data[1:])
            for cycle_idx, d in enumerate(data[1:]):
                print(d[0], d[3], file_name, cycle_idx, 0)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                #print("cycle_idx, d")
                #print(cycle_idx, d)
                classwise_cycle_list.append((d[0],d[1]))
                #print("classwise_cycle_list")
                #print(classwise_cycle_list)

0it [00:00, ?it/s]

136_1b1_Ar_sc_Meditron going on 0 out of 10


1it [00:00,  1.61it/s]

data:-
[-5.36899030e-05 -5.17179863e-04 -2.44551756e-03 ... -1.64040479e-01
 -1.64190946e-01 -1.63977057e-01] 0 136_1b1_Ar_sc_Meditron 0 0
[-5.57448134e-06 -5.33574735e-05 -2.50344552e-04 ...  1.25706215e-01
  1.18222741e-01  1.11026220e-01] 0 136_1b1_Ar_sc_Meditron 1 0
[ 2.13204646e-05  2.05915492e-04  9.77183554e-04 ... -1.30819273e-01
 -1.26692058e-01 -1.22823776e-01] 0 136_1b1_Ar_sc_Meditron 2 0
[-2.18932458e-05 -2.11295211e-04 -1.00147003e-03 ... -5.95232474e-02
 -6.29295678e-02 -6.58988988e-02] 0 136_1b1_Ar_sc_Meditron 3 0
[4.38271136e-07 5.06742033e-06 2.88792401e-05 ... 1.31617094e-01
 1.77430281e-01 2.20164992e-01] 0 136_1b1_Ar_sc_Meditron 4 0
[ 5.20454228e-05  5.06544751e-04  2.42594353e-03 ... -1.38790330e-01
 -1.42331811e-01 -1.45404617e-01] 0 136_1b1_Ar_sc_Meditron 5 0
[ 1.52728380e-05  1.47131317e-04  6.95863159e-04 ... -1.68514520e-01
 -1.52203977e-01 -1.35541847e-01] 0 136_1b1_Ar_sc_Meditron 6 0
[-1.10370906e-04 -1.06606682e-03 -5.05868191e-03 ... -1.29059272e-01
 -1.30

8it [00:01,  6.63it/s]

data:-
[ 8.18797127e-06  7.89951995e-05  3.74071516e-04 ... -1.00748517e-02
 -9.54261246e-03 -8.99811504e-03] 0 143_1b1_Al_sc_Meditron 0 0
[-1.10497847e-05 -1.06670019e-04 -5.05903840e-04 ... -3.68663097e-02
 -3.15026793e-02 -2.60663201e-02] 0 143_1b1_Al_sc_Meditron 1 0
[-8.07878827e-07 -7.18692022e-06 -3.05086368e-05 ...  1.54435641e-02
  1.55066787e-02  1.55663393e-02] 0 143_1b1_Al_sc_Meditron 2 0
[ 1.67159427e-06  1.55773291e-05  7.12583401e-05 ... -8.26113208e-03
 -8.63873062e-03 -8.87038426e-03] 0 143_1b1_Al_sc_Meditron 3 0
[-2.66948465e-06 -2.58000665e-05 -1.22493016e-04 ... -3.86761463e-03
 -4.33285371e-03 -4.82863280e-03] 0 143_1b1_Al_sc_Meditron 4 0
[-6.92205231e-06 -6.69131575e-05 -3.17901916e-04 ...  8.51371317e-02
  8.65809990e-02  8.79709465e-02] 0 143_1b1_Al_sc_Meditron 5 0
[-3.37419893e-05 -3.25912520e-04 -1.54653494e-03 ... -1.57348322e-02
 -1.89180249e-02 -2.20812766e-02] 0 143_1b1_Al_sc_Meditron 6 0
[ 1.81461321e-05  1.75322562e-04  8.32215927e-04 ... -6.72054290e-02


9it [00:02,  4.36it/s]

data:-
[ 3.68380543e-06  3.60081907e-05  1.73393651e-04 ... -4.60296983e-02
 -4.59118572e-02 -4.57112183e-02] 0 144_1b1_Al_sc_Meditron 0 0
[2.66876283e-06 2.57153257e-05 1.21748149e-04 ... 1.03229521e-02
 1.05057699e-02 1.05424531e-02] 0 144_1b1_Al_sc_Meditron 1 0
[-1.10656747e-05 -1.07084213e-04 -5.09253777e-04 ...  2.03608617e-02
  2.03763111e-02  2.04046162e-02] 0 144_1b1_Al_sc_Meditron 2 0
[ 3.67646806e-06  3.56501217e-05  1.69889163e-04 ... -3.15951058e-02
 -3.20933591e-02 -3.26764542e-02] 0 144_1b1_Al_sc_Meditron 3 0
[-1.32876185e-05 -1.28260137e-04 -6.07977048e-04 ... -2.16235790e-02
 -2.14071055e-02 -2.10012218e-02] 0 144_1b1_Al_sc_Meditron 4 0
[-2.84676382e-06 -2.75024066e-05 -1.30621514e-04 ...  2.48191104e-02
  2.49615263e-02  2.51351523e-02] 0 144_1b1_Al_sc_Meditron 5 0
[ 1.13496828e-05  1.09834724e-04  5.22520052e-04 ... -1.23344129e-04
  9.74084909e-05  2.67962123e-04] 0 144_1b1_Al_sc_Meditron 6 0
[2.74864917e-06 2.65359995e-05 1.25846112e-04 ... 9.40458794e-02
 9.4645529

10it [00:02,  3.79it/s]

data:-
[-6.02261937e-06 -5.61490763e-05 -2.54121490e-04 ...  5.03412412e-02
  4.98444752e-02  4.92601282e-02] 0 144_1b1_Tc_sc_Meditron 0 0
[-4.37950158e-06 -4.20576194e-05 -1.98281878e-04 ...  2.23349219e-02
  2.30713244e-02  2.38037946e-02] 0 144_1b1_Tc_sc_Meditron 1 0
[-1.16870744e-05 -1.12932012e-04 -5.35983487e-04 ... -1.83175224e-02
 -1.79065957e-02 -1.74932369e-02] 0 144_1b1_Tc_sc_Meditron 2 0
[-9.87750954e-06 -9.51248253e-05 -4.49686752e-04 ...  2.23852418e-02
  2.21912189e-02  2.19801192e-02] 0 144_1b1_Tc_sc_Meditron 3 0
[-1.31220774e-06 -1.26269195e-05 -5.96277947e-05 ... -1.74712910e-02
 -1.96701311e-02 -2.19601488e-02] 0 144_1b1_Tc_sc_Meditron 4 0
[-1.29137456e-05 -1.25475560e-04 -5.99800932e-04 ... -1.73267926e-02
 -1.67455024e-02 -1.62866746e-02] 0 144_1b1_Tc_sc_Meditron 5 0
[3.31919121e-08 2.59276537e-07 7.19727191e-07 ... 7.45919088e-02
 7.40804066e-02 7.35681091e-02] 0 144_1b1_Tc_sc_Meditron 6 0
[ 2.63901683e-06  2.58894443e-05  1.25356400e-04 ... -8.15060636e-03
 -8.03

In [29]:
classwise_cycle_list[0][1]

0

In [23]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [26]:
#creating sliced and padded data = audio_data
stft_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            stft=librosa.stft(sample[0], n_fft=255, hop_length = 512)
            print(stft.shape, sample[0].shape)
            stft_data.append(padding(stft, 128, 50))

0 out of 93
(128, 17) (8572,)
1 out of 93
(128, 19) (9256,)
2 out of 93
(128, 20) (10232,)
3 out of 93
(128, 18) (8740,)
4 out of 93
(128, 17) (8516,)
5 out of 93
(128, 16) (7820,)
6 out of 93
(128, 18) (8964,)
7 out of 93
(128, 19) (9292,)
8 out of 93
(128, 16) (8188,)
9 out of 93
(128, 22) (10842,)
10 out of 93
(128, 10) (4712,)
11 out of 93
(128, 12) (6079,)
12 out of 93
(128, 10) (4940,)
13 out of 93
(128, 12) (5902,)
14 out of 93
(128, 12) (6006,)
15 out of 93
(128, 13) (6356,)
16 out of 93
(128, 12) (5752,)
17 out of 93
(128, 32) (16364,)
18 out of 93
(128, 11) (5420,)
19 out of 93
(128, 15) (7237,)
20 out of 93
(128, 12) (5750,)
21 out of 93
(128, 15) (7193,)
22 out of 93
(128, 14) (6909,)
23 out of 93
(128, 15) (7498,)
24 out of 93
(128, 16) (7872,)
25 out of 93
(128, 11) (5488,)
26 out of 93
(128, 26) (13248,)
27 out of 93
(128, 9) (4548,)
28 out of 93
(128, 16) (7909,)
29 out of 93
(128, 14) (6994,)
30 out of 93
(128, 16) (7802,)
31 out of 93
(128, 15) (7182,)
32 out of 93
(1

In [30]:
train_dataset=[]
for index in range(len(stft_data)):
    stft=stft_data[index]
    label = classwise_cycle_list[index][1]
    train_dataset.append((stft, label))

In [34]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
    
np.save('X_test_STFT', np.concatenate(X_train, axis=0).reshape(len(train_dataset), 192, 753, 3))
np.save('y_test_STFT', y_train)